In [1]:
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
import pandas as pd
import requests

In [ ]:
# Coloca tus credenciales aquí
CLIENT_ID = ''
CLIENT_SECRET = ''
# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
# Coloca tus credenciales de lastfm aquí
SHARED_SECRET = ''
API_KEY = ''

last_fm_connect = requests.get(f"http://www.last.fm/api/auth/?api_key={API_KEY}")

In [5]:
def extraccion_por_paises(diccionario_zonas, años):
    todos_los_datos = [] # Esto es una lista de dicts donde se van guardando los diccionarios de cada país
    for pais in diccionario_zonas.keys():
        datos_rango = sp.search(q = f'year:{años}', limit=50, offset=0, type='track', market={pais}) # Cada una tiene que cambiar en q el rango de años
        datos_por_pais = {}
        datos_por_pais['pais'] = pais
        datos_por_pais['datos'] = datos_rango
        todos_los_datos.append(datos_por_pais)
    return todos_los_datos
   

In [6]:
def filtro_de_datos(lista_datos_zonas, datos):
    for datos_pais in lista_datos_zonas:
        for element in datos_pais['datos']['tracks']['items']:
            datos['pais'].append(datos_pais['pais'])
            datos['popularity'].append(element['popularity'])
            datos['nombre'].append(element['name'])
            artistas = []
            for artista in element['artists']:
                artistas.append(artista['name'])   
            artistas_str = ', '.join(artistas)
            datos['artista'].append(artistas_str)

In [7]:
paises_norte_europa = {
    "DK": "Dinamarca",
    "EE": "Estonia",
    "FI": "Finlandia",
    "IS": "Islandia",
    "IE": "Irlanda",
    "LT": "Lituania",
    "LV": "Letonia",
    "NO": "Noruega",
    "SE": "Suecia",
    "GB": "Reino Unido"}

In [8]:
p_norte = extraccion_por_paises(paises_norte_europa, '2000-2004')

In [9]:
dic_norte = {'nombre': [], 'popularity': [], 'artista': [], 'pais': [], 'genero': []}

In [10]:
filtro_de_datos(p_norte, dic_norte)

In [11]:
dic_norte

{'nombre': ['Unwritten',
  'Angel',
  'En Dag Tilbage',
  'He Loves U Not',
  'Yellow',
  'Rooftop',
  'Hot!',
  'This Is My Time',
  'Mockingbird',
  'Turn It into Something Special',
  'Drømmer Jeg',
  'Stuck in the Middle with You',
  'Without Me',
  'La vita è',
  'The Scientist',
  'This Is Me',
  'All Falls Down',
  'Handel: The Arrival of the Queen of Sheba (Sinfonia from the oratorio "Solomon" HWV 67)',
  'Numb',
  'Take Me Home',
  'Kinky Fætter',
  'Cabelo Raspadinho',
  'In the End',
  'Devolva-Me',
  'How You Remind Me',
  'Dímelo bajito',
  'The Real Slim Shady',
  'Dar A Volta No Brasil - Ao Vivo',
  'Yeah! (feat. Lil Jon & Ludacris)',
  '2 Faced',
  'Till I Collapse',
  'Kiedy góral umiera',
  'Lækker',
  'San Pedro',
  'Lose Yourself',
  'El peligro',
  'Sparks',
  'Here She Comes Again',
  'In Da Club',
  "American Life (Director's Cut) - 4K Remaster",
  'Murder On The Dancefloor',
  'Beautiful Inside',
  'Ms. Jackson',
  'Wierz? w siebie',
  'Somewhere Only We Know',


In [16]:
lista_nombres_norte = list(dic_norte.values())[0]
lista_artistas_norte = list(dic_norte.values())[2]

In [15]:
def llamada_lastfm(lista_nombres, lista_artistas, datos):
    for i in range(len(lista_nombres)):
        nombre = str(lista_nombres[i]).replace(' ', '+')
        artista = lista_artistas[i].replace(' ', '+').replace(',', '+&')
        llamada_genero = requests.get(f"https://ws.audioscrobbler.com/2.0/?method=track.gettoptags&artist={artista}&track={nombre}&api_key={API_KEY}&autocorrect=1&format=json")
        try:
            genero_cancion = llamada_genero.json()['toptags']['tag'][0]['name']
            datos['genero'].append(genero_cancion)
        except KeyError:
            datos['genero'].append("NULL")
            print(f"La canción {lista_nombres[i]} no está registrada en lastfm")
            
        except IndexError:
            datos['genero'].append("NULL")
            print(f"La canción {lista_nombres[i]} no tiene tags")

In [17]:
llamada_lastfm(lista_nombres_norte, lista_artistas_norte, dic_norte)

La canción Angel no está registrada en lastfm
La canción En Dag Tilbage no está registrada en lastfm
La canción Hot! no está registrada en lastfm
La canción Drømmer Jeg no está registrada en lastfm
La canción All Falls Down no tiene tags
La canción Handel: The Arrival of the Queen of Sheba (Sinfonia from the oratorio "Solomon" HWV 67) no está registrada en lastfm
La canción Take Me Home no está registrada en lastfm
La canción Dímelo bajito no tiene tags
La canción Dar A Volta No Brasil - Ao Vivo no tiene tags
La canción Yeah! (feat. Lil Jon & Ludacris) no está registrada en lastfm
La canción Till I Collapse no está registrada en lastfm
La canción Lækker no está registrada en lastfm
La canción San Pedro no está registrada en lastfm
La canción American Life (Director's Cut) - 4K Remaster no tiene tags
La canción Wierz? w siebie no tiene tags
La canción Love Is Alive - Live; 2004 Remaster no está registrada en lastfm
La canción Ni na no no tiene tags
La canción Angel no está registrada en

In [19]:
df_norte = pd.DataFrame(dic_norte)

In [20]:
df_norte.index.name="id"

In [21]:
df_norte

,nombre,popularity,artista,pais,genero
id,,,,,
0,Unwritten,81,Natasha Bedingfield,DK,pop
1,Angel,28,"Shaggy, Rayvon",DK,NULL
2,En Dag Tilbage,47,Nik & Jay,DK,NULL
3,He Loves U Not,0,Dream,DK,pop
4,Yellow,86,Coldplay,DK,rock
...,...,...,...,...,...
495,Sing For The Moment - Edited,0,Eminem,GB,NULL
496,Bring Me To Life,80,Evanescence,GB,rock
497,Communication Breakdown - Live at The Royal Al...,0,Led Zeppelin,GB,NULL


In [22]:
df_norte.to_csv("paises_norte_2000_2004.csv")